In [43]:
# data manipulation imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# ml imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [75]:
# read in data
train=pd.read_csv('/Users/srikarprayaga/Desktop/NW-Modeling-Assessment/TRAIN_SET_2021.csv')
test=pd.read_csv('/Users/srikarprayaga/Desktop/NW-Modeling-Assessment/TEST_SET_2021.csv')

## Data Cleaning and  Preparation 

In [13]:
train.head()

,RESTAURANT_SERIAL_NUMBER,RESTAURANT_PERMIT_NUMBER,RESTAURANT_NAME,RESTAURANT_LOCATION,RESTAURANT_CATEGORY,ADDRESS,CITY,STATE,ZIP,CURRENT_DEMERITS,...,RECORD_UPDATED,LAT_LONG_RAW,FIRST_VIOLATION,SECOND_VIOLATION,THIRD_VIOLATION,FIRST_VIOLATION_TYPE,SECOND_VIOLATION_TYPE,THIRD_VIOLATION_TYPE,NUMBER_OF_VIOLATIONS,NEXT_INSPECTION_GRADE_C_OR_BELOW
0,DA1117270,PR0004527,SANDS EXPO HALL B2 CONCESSION,SANDS EXPO & CONVENTION CENTER,Snack Bar,201 E Sands Ave,Las Vegas,Nevada,89169-2617,0.0,...,1/6/2015 16:34,"(36.1221442, 115.1652749)",204.0,215.0,302.0,Critical,Major,Imminent Health Hazard,4,0
1,DA1014948,PR0024221,THAI NOODLES CAFE - RESTAURANT,THAI NOODLES CAFE,Restaurant,6710 W CHEYENNE Ave,Las Vegas,Nevada,89108-4588,3.0,...,2/21/2013 22:26,"(36.218691, 115.242913)",214.0,216.0,229.0,Major,Major,Non-Major,5,0
2,DA0861994,PR0019017,SANTA FE SPORTS BOOK GRILL,Santa Fe Station Hotel & Casino,Restaurant,4949 N Rancho Dr,Las Vegas,Nevada,89130-3505,8.0,...,4/6/2015 9:00,"(36.2498922, 115.2441383)",202.0,215.0,233.0,Critical,Major,Non-Major,3,1
3,DA0896719,PR0001343,"Bracken, Walter Elem School Kit","Bracken, Walter Elem School",Elementary School Kitchen,1200 N 27Th St,Las Vegas,Nevada,89101-1517,0.0,...,2/21/2013 22:26,"(36.183669, 115.112355)",214.0,230.0,233.0,Major,Non-Major,Non-Major,3,0
4,DA1031041,PR0006084,HARD ROCK JOINT EAST SVC BAR FL 1,HARD ROCK HOTEL & CASINO,Bar / Tavern,4455 S Paradise Rd,Las Vegas,Nevada,89169-6574,0.0,...,2/21/2013 22:26,"(36.1100828, 115.1538714)",214.0,230.0,233.0,Major,Non-Major,Non-Major,NaN,0


Deal with null values

In [37]:
# check number of null values per column
train.isnull().sum()

RESTAURANT_SERIAL_NUMBER              0
RESTAURANT_PERMIT_NUMBER              0
RESTAURANT_NAME                      65
RESTAURANT_LOCATION                 200
RESTAURANT_CATEGORY                 130
ADDRESS                              70
CITY                                236
STATE                               209
ZIP                                  59
CURRENT_DEMERITS                    216
CURRENT_GRADE                       308
EMPLOYEE_COUNT                       93
MEDIAN_EMPLOYEE_AGE                  34
MEDIAN_EMPLOYEE_TENURE              297
INSPECTION_TIME                     183
INSPECTION_TYPE                     221
INSPECTION_DEMERITS                 254
VIOLATIONS_RAW                      165
RECORD_UPDATED                      119
LAT_LONG_RAW                         15
FIRST_VIOLATION                     212
SECOND_VIOLATION                     85
THIRD_VIOLATION                      61
FIRST_VIOLATION_TYPE                146
SECOND_VIOLATION_TYPE               267


In [22]:
# perentage of null values per column
print(train.isnull().sum()/train.shape[0])
# print average of the null value percentages 
print(np.mean(train.isnull().sum()/train.shape[0])*100)

RESTAURANT_SERIAL_NUMBER            0.000000
RESTAURANT_PERMIT_NUMBER            0.000000
RESTAURANT_NAME                     0.004147
RESTAURANT_LOCATION                 0.012761
RESTAURANT_CATEGORY                 0.008295
ADDRESS                             0.004466
CITY                                0.015058
STATE                               0.013335
ZIP                                 0.003764
CURRENT_DEMERITS                    0.013782
CURRENT_GRADE                       0.019652
EMPLOYEE_COUNT                      0.005934
MEDIAN_EMPLOYEE_AGE                 0.002169
MEDIAN_EMPLOYEE_TENURE              0.018950
INSPECTION_TIME                     0.011676
INSPECTION_TYPE                     0.014101
INSPECTION_DEMERITS                 0.016206
VIOLATIONS_RAW                      0.010528
RECORD_UPDATED                      0.007593
LAT_LONG_RAW                        0.000957
FIRST_VIOLATION                     0.013526
SECOND_VIOLATION                    0.005423
THIRD_VIOL

In [76]:
# drop all null value since null values make up a very small percentage of the data (< 1%)
train.dropna(inplace=True)

In [24]:
# perentage of null values per column after dropping null values
print(train.isnull().sum()/train.shape[0])

RESTAURANT_SERIAL_NUMBER            0.0
RESTAURANT_PERMIT_NUMBER            0.0
RESTAURANT_NAME                     0.0
RESTAURANT_LOCATION                 0.0
RESTAURANT_CATEGORY                 0.0
ADDRESS                             0.0
CITY                                0.0
STATE                               0.0
ZIP                                 0.0
CURRENT_DEMERITS                    0.0
CURRENT_GRADE                       0.0
EMPLOYEE_COUNT                      0.0
MEDIAN_EMPLOYEE_AGE                 0.0
MEDIAN_EMPLOYEE_TENURE              0.0
INSPECTION_TIME                     0.0
INSPECTION_TYPE                     0.0
INSPECTION_DEMERITS                 0.0
VIOLATIONS_RAW                      0.0
RECORD_UPDATED                      0.0
LAT_LONG_RAW                        0.0
FIRST_VIOLATION                     0.0
SECOND_VIOLATION                    0.0
THIRD_VIOLATION                     0.0
FIRST_VIOLATION_TYPE                0.0
SECOND_VIOLATION_TYPE               0.0


In [56]:
# shape of the data after dropping null values
train.shape

(12083, 28)

Deal with outliers in every column 

In [27]:
train.head()

,RESTAURANT_SERIAL_NUMBER,RESTAURANT_PERMIT_NUMBER,RESTAURANT_NAME,RESTAURANT_LOCATION,RESTAURANT_CATEGORY,ADDRESS,CITY,STATE,ZIP,CURRENT_DEMERITS,...,RECORD_UPDATED,LAT_LONG_RAW,FIRST_VIOLATION,SECOND_VIOLATION,THIRD_VIOLATION,FIRST_VIOLATION_TYPE,SECOND_VIOLATION_TYPE,THIRD_VIOLATION_TYPE,NUMBER_OF_VIOLATIONS,NEXT_INSPECTION_GRADE_C_OR_BELOW
0,DA1117270,PR0004527,SANDS EXPO HALL B2 CONCESSION,SANDS EXPO & CONVENTION CENTER,Snack Bar,201 E Sands Ave,Las Vegas,Nevada,89169-2617,0.0,...,1/6/2015 16:34,"(36.1221442, 115.1652749)",204.0,215.0,302.0,Critical,Major,Imminent Health Hazard,4,0
1,DA1014948,PR0024221,THAI NOODLES CAFE - RESTAURANT,THAI NOODLES CAFE,Restaurant,6710 W CHEYENNE Ave,Las Vegas,Nevada,89108-4588,3.0,...,2/21/2013 22:26,"(36.218691, 115.242913)",214.0,216.0,229.0,Major,Major,Non-Major,5,0
2,DA0861994,PR0019017,SANTA FE SPORTS BOOK GRILL,Santa Fe Station Hotel & Casino,Restaurant,4949 N Rancho Dr,Las Vegas,Nevada,89130-3505,8.0,...,4/6/2015 9:00,"(36.2498922, 115.2441383)",202.0,215.0,233.0,Critical,Major,Non-Major,3,1
3,DA0896719,PR0001343,"Bracken, Walter Elem School Kit","Bracken, Walter Elem School",Elementary School Kitchen,1200 N 27Th St,Las Vegas,Nevada,89101-1517,0.0,...,2/21/2013 22:26,"(36.183669, 115.112355)",214.0,230.0,233.0,Major,Non-Major,Non-Major,3,0
5,DA1567561,PR0006724,Hotel Brendans Irish Pub,Orleans Hotel and Casino,Bar / Tavern,4500 W Tropicana Ave,Las Vegas,Nevada,89103-5420,3.0,...,9/11/2015 16:59,"(36.1027192, 115.201984)",206.0,211.0,215.0,Critical,Major,Major,7,0


In [57]:
train.dtypes

RESTAURANT_SERIAL_NUMBER             object
RESTAURANT_PERMIT_NUMBER             object
RESTAURANT_NAME                      object
RESTAURANT_LOCATION                  object
RESTAURANT_CATEGORY                  object
ADDRESS                              object
CITY                                 object
STATE                                object
ZIP                                  object
CURRENT_DEMERITS                    float64
CURRENT_GRADE                        object
EMPLOYEE_COUNT                      float64
MEDIAN_EMPLOYEE_AGE                 float64
MEDIAN_EMPLOYEE_TENURE              float64
INSPECTION_TIME                      object
INSPECTION_TYPE                      object
INSPECTION_DEMERITS                  object
VIOLATIONS_RAW                       object
RECORD_UPDATED                       object
LAT_LONG_RAW                         object
FIRST_VIOLATION                     float64
SECOND_VIOLATION                    float64
THIRD_VIOLATION                 

In [59]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(train['RESTAURANT_SERIAL_NUMBER'].value_counts())

DA1146056    1
DA0972581    1
DA1132069    1
DA1559096    1
DA0001131    1
DA0998321    1
DA1560310    1
DA0860901    1
DA0581753    1
DA0963058    1
DA0825270    1
DA1106407    1
DA0405927    1
DA1445287    1
DA1642376    1
DAAFMBPG7    1
DA0317697    1
DA1012462    1
DA0985919    1
DA0928205    1
DA1119994    1
DA0373511    1
DA1178891    1
DA0875850    1
DA0976929    1
DA0624937    1
DA1664538    1
DA0051845    1
DA1450348    1
DA0215027    1
DA0868260    1
DA0888056    1
DA0960910    1
DA0549017    1
DA0865686    1
DA0640469    1
DA1662533    1
DA1522388    1
DA0616559    1
DA0919629    1
DA0975158    1
DA1326008    1
DA0945311    1
DA1033550    1
DAX738QUE    1
DA0404269    1
DA0846078    1
DA1088613    1
DA1456159    1
DAU2QHU00    1
DA0406525    1
DA0627520    1
DA0974136    1
DA1018141    1
DA0864351    1
DA0493989    1
DA0849700    1
DA0871328    1
DA1007383    1
DA0608944    1
DA0999276    1
DA0990751    1
DAE0O0IAG    1
DA0334324    1
DA1178386    1
DA1016467    1
DA0925681 

In [60]:
# remove all rows from train in column RESTAURANT_SERIAL_NUMBER which dont match "\ADA\d{7}" (AD....) regex expresion 
train = train[train["RESTAURANT_SERIAL_NUMBER"].str.contains("\ADA\d{7}")]
train.shape

(11072, 28)

In [73]:
#remove all rows from train in column RESTAURANT_PERMIT_NUMBER which dont match format where frist 2 characters are "PD" then 2 digits "00" and then 5 random digits
train = train[train["RESTAURANT_PERMIT_NUMBER"].str.contains("PD\d{2}00\d{5}")]


In [74]:
train['RESTAURANT_PERMIT_NUMBER'].head(100)

Series([], Name: RESTAURANT_PERMIT_NUMBER, dtype: object)